# Blackcoffer- Data Extraction and Text Analysis

## Importing Libraries

In [1]:
# Requried imports

from bs4 import BeautifulSoup as bs
import json
import numpy as np
import requests
from requests.models import MissingSchema
import spacy
import trafilatura
import pandas as pd
import string
import nltk    
from nltk.corpus import stopwords
import os
import re
from nltk.tokenize import RegexpTokenizer, sent_tokenize
import itertools

## Loading input excel file

In [4]:
# loading input excel file

df = pd.read_excel(r'Input.xlsx', engine='openpyxl')

In [5]:
df

,URL_ID,URL,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,1,https://insights.blackcoffer.com/how-is-login-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,https://insights.blackcoffer.com/how-does-ai-h...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,https://insights.blackcoffer.com/ai-and-its-im...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,https://insights.blackcoffer.com/how-do-deep-l...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,https://insights.blackcoffer.com/how-artificia...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,167,https://insights.blackcoffer.com/role-big-data...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
166,168,https://insights.blackcoffer.com/sales-forecas...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167,169,https://insights.blackcoffer.com/detect-data-e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
168,170,https://insights.blackcoffer.com/data-exfiltra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df= df.iloc[:,:2]

In [7]:
df

,URL_ID,URL
0,1,https://insights.blackcoffer.com/how-is-login-...
1,2,https://insights.blackcoffer.com/how-does-ai-h...
2,3,https://insights.blackcoffer.com/ai-and-its-im...
3,4,https://insights.blackcoffer.com/how-do-deep-l...
4,5,https://insights.blackcoffer.com/how-artificia...
...,...,...
165,167,https://insights.blackcoffer.com/role-big-data...
166,168,https://insights.blackcoffer.com/sales-forecas...
167,169,https://insights.blackcoffer.com/detect-data-e...
168,170,https://insights.blackcoffer.com/data-exfiltra...


In [8]:
# list of urls

urls= list(df["URL"])

In [9]:
urls

['https://insights.blackcoffer.com/how-is-login-logout-time-tracking-for-employees-in-office-done-by-ai/',
 'https://insights.blackcoffer.com/how-does-ai-help-to-monitor-retail-shelf-watches/',
 'https://insights.blackcoffer.com/ai-and-its-impact-on-the-fashion-industry/',
 'https://insights.blackcoffer.com/how-do-deep-learning-models-predict-old-and-new-drugs-that-are-successfully-treated-in-healthcare/',
 'https://insights.blackcoffer.com/how-artificial-intelligence-can-boost-your-productivity-level/',
 'https://insights.blackcoffer.com/how-are-genetic-sequencing-maps-affected-by-deep-learning-and-ai/',
 'https://insights.blackcoffer.com/how-is-ai-used-to-solve-traffic-management/',
 'https://insights.blackcoffer.com/benefits-of-big-data-in-different-fields/',
 'https://insights.blackcoffer.com/how-big-data-will-impact-the-future-of-business/',
 'https://insights.blackcoffer.com/how-will-ai-make-decisions-in-tomorrows-wars/',
 'https://insights.blackcoffer.com/which-one-is-better-ai-

## Extracting Text Title from Webpages

In [10]:
# extracting text title from webpages

title=[]

for i in urls:
    headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"
    }
    page = requests.get(i, headers=headers)
    soup = bs(page.content, 'html.parser')
    soup.prettify()
    
    title.append(soup.find_all('title'))

In [11]:
# joining extracted title to dataframe

l= pd.Series(title)
df['Title']= l.values

In [12]:
df

,URL_ID,URL,Title
0,1,https://insights.blackcoffer.com/how-is-login-...,[[How is Login Logout Time Tracking for Employ...
1,2,https://insights.blackcoffer.com/how-does-ai-h...,[[How does AI help to monitor Retail Shelf wat...
2,3,https://insights.blackcoffer.com/ai-and-its-im...,[[AI and its impact on the Fashion Industry - ...
3,4,https://insights.blackcoffer.com/how-do-deep-l...,[[How do deep learning models predict old and ...
4,5,https://insights.blackcoffer.com/how-artificia...,[[How artificial intelligence can boost your p...
...,...,...,...
165,167,https://insights.blackcoffer.com/role-big-data...,[[Role of Big Data in Academia - Blackcoffer I...
166,168,https://insights.blackcoffer.com/sales-forecas...,[[Methods for Sales Forecasting in Retail - Bl...
167,169,https://insights.blackcoffer.com/detect-data-e...,[[Detect Data ExFiltration over the Network - ...
168,170,https://insights.blackcoffer.com/data-exfiltra...,[[What is Data Exfiltration? - Blackcoffer Ins...


In [62]:
# cleaning title text

def return_titles(a):
    return a.text

In [63]:
df['Title']= df['Title'].apply(return_titles)

In [64]:
df

,URL_ID,URL,Title,Content,Content String,Length Content String,stopword_removed_content,length_stopword_removed_content,POSITIVE SCORE,NEGATIVE SCORE,...,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,1,https://insights.blackcoffer.com/how-is-login-...,How is Login Logout Time Tracking for Employee...,[When people hear AI they often think about se...,When people hear AI they often think about sen...,4286,"[people, hear, sentient, robots, magic, boxes,...",327,2,6,...,0.024465,16,0.388379,6.555352,16,127,327,1239,4,5.234352
1,2,https://insights.blackcoffer.com/how-does-ai-h...,How does AI help to monitor Retail Shelf watch...,[With increasing computing power and more data...,"With increasing computing power and more data,...",3984,"[increasing, computing, data, potential, algor...",303,8,6,...,0.046205,15,0.445545,6.178218,15,135,303,1155,2,5.479675
2,3,https://insights.blackcoffer.com/ai-and-its-im...,AI and its impact on the Fashion Industry - Bl...,[If you were a fan of the 90’s film Clueless b...,If you were a fan of the ’s film Clueless ba...,11324,"[fan, film, clueless, ll, remember, protagonis...",863,31,23,...,0.062572,22,0.483198,8.993279,22,417,863,3231,10,5.619883
3,4,https://insights.blackcoffer.com/how-do-deep-l...,How do deep learning models predict old and ne...,"[Understanding exactly how data is ingested, a...","Understanding exactly how data is ingested, an...",2706,"[understanding, data, ingested, analyzed, retu...",213,3,1,...,0.018779,15,0.502347,6.200939,15,107,213,818,1,5.384434
4,5,https://insights.blackcoffer.com/how-artificia...,How artificial intelligence can boost your pro...,"[From the stone age to the modern world, from ...","From the stone age to the modern world, from h...",7484,"[age, modern, world, hunting, gathering, culti...",527,18,17,...,0.066414,11,0.483871,4.593548,11,255,527,2304,26,5.254788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,167,https://insights.blackcoffer.com/role-big-data...,Role of Big Data in Academia - Blackcoffer Ins...,"[Can academia, researchers, decision makers an...","Can academia, researchers, decision makers and...",9235,"[academia, researchers, decision, makers, poli...",652,15,37,...,0.079755,14,0.446319,5.778528,14,291,652,2740,14,5.300613
166,168,https://insights.blackcoffer.com/sales-forecas...,Methods for Sales Forecasting in Retail - Blac...,[Inventory planning is a fundamental part of r...,Inventory planning is a fundamental part of re...,6132,"[inventory, planning, fundamental, part, retai...",480,18,15,...,0.068750,17,0.477083,6.990833,17,229,480,1745,0,5.722588
167,169,https://insights.blackcoffer.com/detect-data-e...,Detect Data ExFiltration over the Network - Bl...,[Insider threat detection specifically to dete...,Insider threat detection specifically to detec...,6222,"[insider, threat, detection, specifically, det...",520,4,47,...,0.098077,12,0.446154,4.978462,12,232,520,1829,6,5.688242
168,170,https://insights.blackcoffer.com/data-exfiltra...,What is Data Exfiltration? - Blackcoffer Insights,"[If we talk in terms of our general life, Exfi...","If we talk in terms of our general life, Exfil...",2989,"[talk, terms, general, life, exfiltrate, surre...",205,4,8,...,0.058537,10,0.487805,4.195122,10,100,205,900,11,5.118609


## Extracting Content from Webpages

In [16]:
# extracting text 

def new_func(val):
    s=[]
    for i in range(len(val)):
        s.append(val[i].text)
    return s

In [17]:
# extracting content from webpages

content=[]

for i in urls:
    headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"
    }
    page = requests.get(i, headers=headers)
    soup = bs(page.content, 'html.parser')
    soup.prettify()
    a=soup.select(".td-post-content")
    b= a[0]
    
    if b.select('p')!= []:
        content.append(new_func(b.select('p')))
        
        if b.select('li')!= []:
            m= new_func(b.select('li'))
            content[-1].append(m)
        else:
            pass

In [18]:
# joining extracted content to dataframe

c= pd.Series(content)
df['Content']= c.values

In [19]:
df

,URL_ID,URL,Title,Content
0,1,https://insights.blackcoffer.com/how-is-login-...,[How is Login Logout Time Tracking for Employe...,[When people hear AI they often think about se...
1,2,https://insights.blackcoffer.com/how-does-ai-h...,[How does AI help to monitor Retail Shelf watc...,[With increasing computing power and more data...
2,3,https://insights.blackcoffer.com/ai-and-its-im...,[AI and its impact on the Fashion Industry - B...,[If you were a fan of the 90’s film Clueless b...
3,4,https://insights.blackcoffer.com/how-do-deep-l...,[How do deep learning models predict old and n...,"[Understanding exactly how data is ingested, a..."
4,5,https://insights.blackcoffer.com/how-artificia...,[How artificial intelligence can boost your pr...,"[From the stone age to the modern world, from ..."
...,...,...,...,...
165,167,https://insights.blackcoffer.com/role-big-data...,[Role of Big Data in Academia - Blackcoffer In...,"[Can academia, researchers, decision makers an..."
166,168,https://insights.blackcoffer.com/sales-forecas...,[Methods for Sales Forecasting in Retail - Bla...,[Inventory planning is a fundamental part of r...
167,169,https://insights.blackcoffer.com/detect-data-e...,[Detect Data ExFiltration over the Network - B...,[Insider threat detection specifically to dete...
168,170,https://insights.blackcoffer.com/data-exfiltra...,[What is Data Exfiltration? - Blackcoffer Insi...,"[If we talk in terms of our general life, Exfi..."


## Cleaning Extracted Content

In [20]:
# converting to string and removing special characters and numeric 

def func(val):
    a= "".join(list(itertools.chain(*val)))
    pattern = r'[0-9]'
    b = re.sub(pattern, ' ', a)
    c= re.sub("[""'''-()]","",b)
    return c

In [21]:
df["Content String"]= df["Content"].apply(func)

In [65]:
df

,URL_ID,URL,Title,Content,Content String,Length Content String,stopword_removed_content,length_stopword_removed_content,POSITIVE SCORE,NEGATIVE SCORE,...,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,1,https://insights.blackcoffer.com/how-is-login-...,How is Login Logout Time Tracking for Employee...,[When people hear AI they often think about se...,When people hear AI they often think about sen...,4286,"[people, hear, sentient, robots, magic, boxes,...",327,2,6,...,0.024465,16,0.388379,6.555352,16,127,327,1239,4,5.234352
1,2,https://insights.blackcoffer.com/how-does-ai-h...,How does AI help to monitor Retail Shelf watch...,[With increasing computing power and more data...,"With increasing computing power and more data,...",3984,"[increasing, computing, data, potential, algor...",303,8,6,...,0.046205,15,0.445545,6.178218,15,135,303,1155,2,5.479675
2,3,https://insights.blackcoffer.com/ai-and-its-im...,AI and its impact on the Fashion Industry - Bl...,[If you were a fan of the 90’s film Clueless b...,If you were a fan of the ’s film Clueless ba...,11324,"[fan, film, clueless, ll, remember, protagonis...",863,31,23,...,0.062572,22,0.483198,8.993279,22,417,863,3231,10,5.619883
3,4,https://insights.blackcoffer.com/how-do-deep-l...,How do deep learning models predict old and ne...,"[Understanding exactly how data is ingested, a...","Understanding exactly how data is ingested, an...",2706,"[understanding, data, ingested, analyzed, retu...",213,3,1,...,0.018779,15,0.502347,6.200939,15,107,213,818,1,5.384434
4,5,https://insights.blackcoffer.com/how-artificia...,How artificial intelligence can boost your pro...,"[From the stone age to the modern world, from ...","From the stone age to the modern world, from h...",7484,"[age, modern, world, hunting, gathering, culti...",527,18,17,...,0.066414,11,0.483871,4.593548,11,255,527,2304,26,5.254788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,167,https://insights.blackcoffer.com/role-big-data...,Role of Big Data in Academia - Blackcoffer Ins...,"[Can academia, researchers, decision makers an...","Can academia, researchers, decision makers and...",9235,"[academia, researchers, decision, makers, poli...",652,15,37,...,0.079755,14,0.446319,5.778528,14,291,652,2740,14,5.300613
166,168,https://insights.blackcoffer.com/sales-forecas...,Methods for Sales Forecasting in Retail - Blac...,[Inventory planning is a fundamental part of r...,Inventory planning is a fundamental part of re...,6132,"[inventory, planning, fundamental, part, retai...",480,18,15,...,0.068750,17,0.477083,6.990833,17,229,480,1745,0,5.722588
167,169,https://insights.blackcoffer.com/detect-data-e...,Detect Data ExFiltration over the Network - Bl...,[Insider threat detection specifically to dete...,Insider threat detection specifically to detec...,6222,"[insider, threat, detection, specifically, det...",520,4,47,...,0.098077,12,0.446154,4.978462,12,232,520,1829,6,5.688242
168,170,https://insights.blackcoffer.com/data-exfiltra...,What is Data Exfiltration? - Blackcoffer Insights,"[If we talk in terms of our general life, Exfi...","If we talk in terms of our general life, Exfil...",2989,"[talk, terms, general, life, exfiltrate, surre...",205,4,8,...,0.058537,10,0.487805,4.195122,10,100,205,900,11,5.118609


In [23]:
df["Length Content String"]= df["Content String"].apply(len)

## Loading Stopwords File

In [25]:
# loading stopword file

stopword_lists=[]
with open('stopwords.txt') as f:
    for line in f:
        inner_list = [elt.strip() for elt in line.split(',')]
        stopword_lists.append(inner_list)

In [26]:
# converting imported file to list of strings
# merging stopword file to its lowercase and uppercase

merged_list = list(itertools.chain(*stopword_lists))
merged_list_lower= list(map(lambda x: x.lower(), merged_list))
merged_list_upper= list(map(lambda x: x.upper(), merged_list))
new_stopwords_list= merged_list+ merged_list_lower+merged_list_upper

## Filtering Stopword and Special Characters

In [27]:
# removing special characters and stopwords from string
# converting string to list of string

def tokenizer(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = list(filter(lambda token: token not in new_stopwords_list, tokens))
    return filtered_words

In [28]:
df["stopword_removed_content"]= df['Content String'].apply(tokenizer)
df["length_stopword_removed_content"]= df["stopword_removed_content"].apply(len)

In [66]:
df

,URL_ID,URL,Title,Content,Content String,Length Content String,stopword_removed_content,length_stopword_removed_content,POSITIVE SCORE,NEGATIVE SCORE,...,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,1,https://insights.blackcoffer.com/how-is-login-...,How is Login Logout Time Tracking for Employee...,[When people hear AI they often think about se...,When people hear AI they often think about sen...,4286,"[people, hear, sentient, robots, magic, boxes,...",327,2,6,...,0.024465,16,0.388379,6.555352,16,127,327,1239,4,5.234352
1,2,https://insights.blackcoffer.com/how-does-ai-h...,How does AI help to monitor Retail Shelf watch...,[With increasing computing power and more data...,"With increasing computing power and more data,...",3984,"[increasing, computing, data, potential, algor...",303,8,6,...,0.046205,15,0.445545,6.178218,15,135,303,1155,2,5.479675
2,3,https://insights.blackcoffer.com/ai-and-its-im...,AI and its impact on the Fashion Industry - Bl...,[If you were a fan of the 90’s film Clueless b...,If you were a fan of the ’s film Clueless ba...,11324,"[fan, film, clueless, ll, remember, protagonis...",863,31,23,...,0.062572,22,0.483198,8.993279,22,417,863,3231,10,5.619883
3,4,https://insights.blackcoffer.com/how-do-deep-l...,How do deep learning models predict old and ne...,"[Understanding exactly how data is ingested, a...","Understanding exactly how data is ingested, an...",2706,"[understanding, data, ingested, analyzed, retu...",213,3,1,...,0.018779,15,0.502347,6.200939,15,107,213,818,1,5.384434
4,5,https://insights.blackcoffer.com/how-artificia...,How artificial intelligence can boost your pro...,"[From the stone age to the modern world, from ...","From the stone age to the modern world, from h...",7484,"[age, modern, world, hunting, gathering, culti...",527,18,17,...,0.066414,11,0.483871,4.593548,11,255,527,2304,26,5.254788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,167,https://insights.blackcoffer.com/role-big-data...,Role of Big Data in Academia - Blackcoffer Ins...,"[Can academia, researchers, decision makers an...","Can academia, researchers, decision makers and...",9235,"[academia, researchers, decision, makers, poli...",652,15,37,...,0.079755,14,0.446319,5.778528,14,291,652,2740,14,5.300613
166,168,https://insights.blackcoffer.com/sales-forecas...,Methods for Sales Forecasting in Retail - Blac...,[Inventory planning is a fundamental part of r...,Inventory planning is a fundamental part of re...,6132,"[inventory, planning, fundamental, part, retai...",480,18,15,...,0.068750,17,0.477083,6.990833,17,229,480,1745,0,5.722588
167,169,https://insights.blackcoffer.com/detect-data-e...,Detect Data ExFiltration over the Network - Bl...,[Insider threat detection specifically to dete...,Insider threat detection specifically to detec...,6222,"[insider, threat, detection, specifically, det...",520,4,47,...,0.098077,12,0.446154,4.978462,12,232,520,1829,6,5.688242
168,170,https://insights.blackcoffer.com/data-exfiltra...,What is Data Exfiltration? - Blackcoffer Insights,"[If we talk in terms of our general life, Exfi...","If we talk in terms of our general life, Exfil...",2989,"[talk, terms, general, life, exfiltrate, surre...",205,4,8,...,0.058537,10,0.487805,4.195122,10,100,205,900,11,5.118609


## Dictionary of Positive and Negative Words

In [30]:
# Dictionary of positive and negative words

lmdict = {'Negative': ['abandon', 'abandoned', 'abandoning', 'abandonment', 'abandonments', 'abandons', 'abdicated',
                       'abdicates', 'abdicating', 'abdication', 'abdications', 'aberrant', 'aberration', 'aberrational',
                       'aberrations', 'abetting', 'abnormal', 'abnormalities', 'abnormality', 'abnormally', 'abolish',
                       'abolished', 'abolishes', 'abolishing', 'abrogate', 'abrogated', 'abrogates', 'abrogating',
                       'abrogation', 'abrogations', 'abrupt', 'abruptly', 'abruptness', 'absence', 'absences',
                       'absenteeism', 'abuse', 'abused', 'abuses', 'abusing', 'abusive', 'abusively', 'abusiveness',
                       'accident', 'accidental', 'accidentally', 'accidents', 'accusation', 'accusations', 'accuse',
                       'accused', 'accuses', 'accusing', 'acquiesce', 'acquiesced', 'acquiesces', 'acquiescing',
                       'acquit', 'acquits', 'acquittal', 'acquittals', 'acquitted', 'acquitting', 'adulterate',
                       'adulterated', 'adulterating', 'adulteration', 'adulterations', 'adversarial', 'adversaries',
                       'adversary', 'adverse', 'adversely', 'adversities', 'adversity', 'aftermath', 'aftermaths',
                       'against', 'aggravate', 'aggravated', 'aggravates', 'aggravating', 'aggravation', 'aggravations',
                       'alerted', 'alerting', 'alienate', 'alienated', 'alienates', 'alienating', 'alienation',
                       'alienations', 'allegation', 'allegations', 'allege', 'alleged', 'allegedly', 'alleges',
                       'alleging', 'annoy', 'annoyance', 'annoyances', 'annoyed', 'annoying', 'annoys', 'annul',
                       'annulled', 'annulling', 'annulment', 'annulments', 'annuls', 'anomalies', 'anomalous',
                       'anomalously', 'anomaly', 'anticompetitive', 'antitrust', 'argue', 'argued', 'arguing',
                       'argument', 'argumentative', 'arguments', 'arrearage', 'arrearages', 'arrears', 'arrest',
                       'arrested', 'arrests', 'artificially', 'assault', 'assaulted', 'assaulting', 'assaults',
                       'assertions', 'attrition', 'aversely', 'backdating', 'bad', 'bail', 'bailout', 'balk', 'balked',
                       'bankrupt', 'bankruptcies', 'bankruptcy', 'bankrupted', 'bankrupting', 'bankrupts', 'bans',
                       'barred', 'barrier', 'barriers', 'bottleneck', 'bottlenecks', 'boycott', 'boycotted',
                       'boycotting', 'boycotts', 'breach', 'breached', 'breaches', 'breaching', 'break', 'breakage',
                       'breakages', 'breakdown', 'breakdowns', 'breaking', 'breaks', 'bribe', 'bribed', 'briberies',
                       'bribery', 'bribes', 'bribing', 'bridge', 'broken', 'burden', 'burdened', 'burdening', 'burdens',
                       'burdensome', 'burned', 'calamities', 'calamitous', 'calamity', 'cancel', 'canceled',
                       'canceling', 'cancellation', 'cancellations', 'cancelled', 'cancelling', 'cancels', 'careless',
                       'carelessly', 'carelessness', 'catastrophe', 'catastrophes', 'catastrophic', 'catastrophically',
                       'caution', 'cautionary', 'cautioned', 'cautioning', 'cautions', 'cease', 'ceased', 'ceases',
                       'ceasing', 'censure', 'censured', 'censures', 'censuring', 'challenge', 'challenged',
                       'challenges', 'challenging', 'chargeoffs', 'circumvent', 'circumvented', 'circumventing',
                       'circumvention', 'circumventions', 'circumvents', 'claiming', 'claims', 'clawback', 'closed',
                       'closeout', 'closeouts', 'closing', 'closings', 'closure', 'closures', 'coerce', 'coerced',
                       'coerces', 'coercing', 'coercion', 'coercive', 'collapse', 'collapsed', 'collapses',
                       'collapsing', 'collision', 'collisions', 'collude', 'colluded', 'colludes', 'colluding',
                       'collusion', 'collusions', 'collusive', 'complain', 'complained', 'complaining', 'complains',
                       'complaint', 'complaints', 'complicate', 'complicated', 'complicates', 'complicating',
                       'complication', 'complications', 'compulsion', 'concealed', 'concealing', 'concede', 'conceded',
                       'concedes', 'conceding', 'concern', 'concerned', 'concerns', 'conciliating', 'conciliation',
                       'conciliations', 'condemn', 'condemnation', 'condemnations', 'condemned', 'condemning',
                       'condemns', 'condone', 'condoned', 'confess', 'confessed', 'confesses', 'confessing',
                       'confession', 'confine', 'confined', 'confinement', 'confinements', 'confines', 'confining',
                       'confiscate', 'confiscated', 'confiscates', 'confiscating', 'confiscation', 'confiscations',
                       'conflict', 'conflicted', 'conflicting', 'conflicts', 'confront', 'confrontation',
                       'confrontational', 'confrontations', 'confronted', 'confronting', 'confronts', 'confuse',
                       'confused', 'confuses', 'confusing', 'confusingly', 'confusion', 'conspiracies', 'conspiracy',
                       'conspirator', 'conspiratorial', 'conspirators', 'conspire', 'conspired', 'conspires',
                       'conspiring', 'contempt', 'contend', 'contended', 'contending', 'contends', 'contention',
                       'contentions', 'contentious', 'contentiously', 'contested', 'contesting', 'contraction',
                       'contractions', 'contradict', 'contradicted', 'contradicting', 'contradiction', 'contradictions',
                       'contradictory', 'contradicts', 'contrary', 'controversial', 'controversies', 'controversy',
                       'convict', 'convicted', 'convicting', 'conviction', 'convictions', 'corrected', 'correcting',
                       'correction', 'corrections', 'corrects', 'corrupt', 'corrupted', 'corrupting', 'corruption',
                       'corruptions', 'corruptly', 'corruptness', 'costly', 'counterclaim', 'counterclaimed',
                       'counterclaiming', 'counterclaims', 'counterfeit', 'counterfeited', 'counterfeiter',
                       'counterfeiters', 'counterfeiting', 'counterfeits', 'countermeasure', 'countermeasures', 'crime',
                       'crimes', 'criminal', 'criminally', 'criminals', 'crises', 'crisis', 'critical', 'critically',
                       'criticism', 'criticisms', 'criticize', 'criticized', 'criticizes', 'criticizing', 'crucial',
                       'crucially', 'culpability', 'culpable', 'culpably', 'cumbersome', 'curtail', 'curtailed',
                       'curtailing', 'curtailment', 'curtailments', 'curtails', 'cut', 'cutback', 'cutbacks',
                       'cyberattack', 'cyberattacks', 'cyberbullying', 'cybercrime', 'cybercrimes', 'cybercriminal',
                       'cybercriminals', 'damage', 'damaged', 'damages', 'damaging', 'dampen', 'dampened', 'danger',
                       'dangerous', 'dangerously', 'dangers', 'deadlock', 'deadlocked', 'deadlocking', 'deadlocks',
                       'deadweight', 'deadweights', 'debarment', 'debarments', 'debarred', 'deceased', 'deceit',
                       'deceitful', 'deceitfulness', 'deceive', 'deceived', 'deceives', 'deceiving', 'deception',
                       'deceptions', 'deceptive', 'deceptively', 'decline', 'declined', 'declines', 'declining',
                       'deface', 'defaced', 'defacement', 'defamation', 'defamations', 'defamatory', 'defame',
                       'defamed', 'defames', 'defaming', 'default', 'defaulted', 'defaulting', 'defaults', 'defeat',
                       'defeated', 'defeating', 'defeats', 'defect', 'defective', 'defects', 'defend', 'defendant',
                       'defendants', 'defended', 'defending', 'defends', 'defensive', 'defer', 'deficiencies',
                       'deficiency', 'deficient', 'deficit', 'deficits', 'defraud', 'defrauded', 'defrauding',
                       'defrauds', 'defunct', 'degradation', 'degradations', 'degrade', 'degraded', 'degrades',
                       'degrading', 'delay', 'delayed', 'delaying', 'delays', 'deleterious', 'deliberate',
                       'deliberated', 'deliberately', 'delinquencies', 'delinquency', 'delinquent', 'delinquently',
                       'delinquents', 'delist', 'delisted', 'delisting', 'delists', 'demise', 'demised', 'demises',
                       'demising', 'demolish', 'demolished', 'demolishes', 'demolishing', 'demolition', 'demolitions',
                       'demote', 'demoted', 'demotes', 'demoting', 'demotion', 'demotions', 'denial', 'denials',
                       'denied', 'denies', 'denigrate', 'denigrated', 'denigrates', 'denigrating', 'denigration',
                       'deny', 'denying', 'deplete', 'depleted', 'depletes', 'depleting', 'depletion', 'depletions',
                       'deprecation', 'depress', 'depressed', 'depresses', 'depressing', 'deprivation', 'deprive',
                       'deprived', 'deprives', 'depriving', 'derelict', 'dereliction', 'derogatory', 'destabilization',
                       'destabilize', 'destabilized', 'destabilizing', 'destroy', 'destroyed', 'destroying', 'destroys',
                       'destruction', 'destructive', 'detain', 'detained', 'detention', 'detentions', 'deter',
                       'deteriorate', 'deteriorated', 'deteriorates', 'deteriorating', 'deterioration',
                       'deteriorations', 'deterred', 'deterrence', 'deterrences', 'deterrent', 'deterrents',
                       'deterring', 'deters', 'detract', 'detracted', 'detracting', 'detriment', 'detrimental',
                       'detrimentally', 'detriments', 'devalue', 'devalued', 'devalues', 'devaluing', 'devastate',
                       'devastated', 'devastating', 'devastation', 'deviate', 'deviated', 'deviates', 'deviating',
                       'deviation', 'deviations', 'devolve', 'devolved', 'devolves', 'devolving', 'difficult',
                       'difficulties', 'difficultly', 'difficulty', 'diminish', 'diminished', 'diminishes',
                       'diminishing', 'diminution', 'disadvantage', 'disadvantaged', 'disadvantageous', 'disadvantages',
                       'disaffiliation', 'disagree', 'disagreeable', 'disagreed', 'disagreeing', 'disagreement',
                       'disagreements', 'disagrees', 'disallow', 'disallowance', 'disallowances', 'disallowed',
                       'disallowing', 'disallows', 'disappear', 'disappearance', 'disappearances', 'disappeared',
                       'disappearing', 'disappears', 'disappoint', 'disappointed', 'disappointing', 'disappointingly',
                       'disappointment', 'disappointments', 'disappoints', 'disapproval', 'disapprovals', 'disapprove',
                       'disapproved', 'disapproves', 'disapproving', 'disassociates', 'disassociating',
                       'disassociation', 'disassociations', 'disaster', 'disasters', 'disastrous', 'disastrously',
                       'disavow', 'disavowal', 'disavowed', 'disavowing', 'disavows', 'disciplinary', 'disclaim',
                       'disclaimed', 'disclaimer', 'disclaimers', 'disclaiming', 'disclaims', 'disclose', 'disclosed',
                       'discloses', 'disclosing', 'discontinuance', 'discontinuances', 'discontinuation',
                       'discontinuations', 'discontinue', 'discontinued', 'discontinues', 'discontinuing', 'discourage',
                       'discouraged', 'discourages', 'discouraging', 'discredit', 'discredited', 'discrediting',
                       'discredits', 'discrepancies', 'discrepancy', 'disfavor', 'disfavored', 'disfavoring',
                       'disfavors', 'disgorge', 'disgorged', 'disgorgement', 'disgorgements', 'disgorges', 'disgorging',
                       'disgrace', 'disgraceful', 'disgracefully', 'dishonest', 'dishonestly', 'dishonesty', 'dishonor',
                       'dishonorable', 'dishonorably', 'dishonored', 'dishonoring', 'dishonors', 'disincentives',
                       'disinterested', 'disinterestedly', 'disinterestedness', 'disloyal', 'disloyally', 'disloyalty',
                       'dismal', 'dismally', 'dismiss', 'dismissal', 'dismissals', 'dismissed', 'dismisses',
                       'dismissing', 'disorderly', 'disparage', 'disparaged', 'disparagement', 'disparagements',
                       'disparages', 'disparaging', 'disparagingly', 'disparities', 'disparity', 'displace',
                       'displaced', 'displacement', 'displacements', 'displaces', 'displacing', 'dispose', 'dispossess',
                       'dispossessed', 'dispossesses', 'dispossessing', 'disproportion', 'disproportional',
                       'disproportionate', 'disproportionately', 'dispute', 'disputed', 'disputes', 'disputing',
                       'disqualification', 'disqualifications', 'disqualified', 'disqualifies', 'disqualify',
                       'disqualifying', 'disregard', 'disregarded', 'disregarding', 'disregards', 'disreputable',
                       'disrepute', 'disrupt', 'disrupted', 'disrupting', 'disruption', 'disruptions', 'disruptive',
                       'disrupts', 'dissatisfaction', 'dissatisfied', 'dissent', 'dissented', 'dissenter', 'dissenters',
                       'dissenting', 'dissents', 'dissident', 'dissidents', 'dissolution', 'dissolutions', 'distort',
                       'distorted', 'distorting', 'distortion', 'distortions', 'distorts', 'distract', 'distracted',
                       'distracting', 'distraction', 'distractions', 'distracts', 'distress', 'distressed', 'disturb',
                       'disturbance', 'disturbances', 'disturbed', 'disturbing', 'disturbs', 'diversion', 'divert',
                       'diverted', 'diverting', 'diverts', 'divest', 'divested', 'divesting', 'divestiture',
                       'divestitures', 'divestment', 'divestments', 'divests', 'divorce', 'divorced', 'divulge',
                       'divulged', 'divulges', 'divulging', 'doubt', 'doubted', 'doubtful', 'doubts', 'downgrade',
                       'downgraded', 'downgrades', 'downgrading', 'downsize', 'downsized', 'downsizes', 'downsizing',
                       'downsizings', 'downtime', 'downtimes', 'downturn', 'downturns', 'downward', 'downwards', 'drag',
                       'drastic', 'drastically', 'drawback', 'drawbacks', 'dropped', 'drought', 'droughts', 'duress',
                       'dysfunction', 'dysfunctional', 'dysfunctions', 'easing', 'egregious', 'egregiously', 'embargo',
                       'embargoed', 'embargoes', 'embargoing', 'embarrass', 'embarrassed', 'embarrasses',
                       'embarrassing', 'embarrassment', 'embarrassments', 'embezzle', 'embezzled', 'embezzlement',
                       'embezzlements', 'embezzler', 'embezzles', 'embezzling', 'encroach', 'encroached', 'encroaches',
                       'encroaching', 'encroachment', 'encroachments', 'encumber', 'encumbered', 'encumbering',
                       'encumbers', 'encumbrance', 'encumbrances', 'endanger', 'endangered', 'endangering',
                       'endangerment', 'endangers', 'enjoin', 'enjoined', 'enjoining', 'enjoins', 'erode', 'eroded',
                       'erodes', 'eroding', 'erosion', 'erratic', 'erratically', 'erred', 'erring', 'erroneous',
                       'erroneously', 'error', 'errors', 'errs', 'escalate', 'escalated', 'escalates', 'escalating',
                       'evade', 'evaded', 'evades', 'evading', 'evasion', 'evasions', 'evasive', 'evict', 'evicted',
                       'evicting', 'eviction', 'evictions', 'evicts', 'exacerbate', 'exacerbated', 'exacerbates',
                       'exacerbating', 'exacerbation', 'exacerbations', 'exaggerate', 'exaggerated', 'exaggerates',
                       'exaggerating', 'exaggeration', 'excessive', 'excessively', 'exculpate', 'exculpated',
                       'exculpates', 'exculpating', 'exculpation', 'exculpations', 'exculpatory', 'exonerate',
                       'exonerated', 'exonerates', 'exonerating', 'exoneration', 'exonerations', 'exploit',
                       'exploitation', 'exploitations', 'exploitative', 'exploited', 'exploiting', 'exploits', 'expose',
                       'exposed', 'exposes', 'exposing', 'expropriate', 'expropriated', 'expropriates', 'expropriating',
                       'expropriation', 'expropriations', 'expulsion', 'expulsions', 'extenuating', 'fail', 'failed',
                       'failing', 'failings', 'fails', 'failure', 'failures', 'fallout', 'false', 'falsely',
                       'falsification', 'falsifications', 'falsified', 'falsifies', 'falsify', 'falsifying', 'falsity',
                       'fatalities', 'fatality', 'fatally', 'fault', 'faulted', 'faults', 'faulty', 'fear', 'fears',
                       'felonies', 'felonious', 'felony', 'fictitious', 'fined', 'fines', 'fired', 'firing', 'flaw',
                       'flawed', 'flaws', 'forbid', 'forbidden', 'forbidding', 'forbids', 'force', 'forced', 'forcing',
                       'foreclose', 'foreclosed', 'forecloses', 'foreclosing', 'foreclosure', 'foreclosures', 'forego',
                       'foregoes', 'foregone', 'forestall', 'forestalled', 'forestalling', 'forestalls', 'forfeit',
                       'forfeited', 'forfeiting', 'forfeits', 'forfeiture', 'forfeitures', 'forgers', 'forgery',
                       'fraud', 'frauds', 'fraudulence', 'fraudulent', 'fraudulently', 'frivolous', 'frivolously',
                       'frustrate', 'frustrated', 'frustrates', 'frustrating', 'frustratingly', 'frustration',
                       'frustrations', 'fugitive', 'fugitives', 'gratuitous', 'gratuitously', 'grievance', 'grievances',
                       'grossly', 'groundless', 'guilty', 'halt', 'halted', 'hamper', 'hampered', 'hampering',
                       'hampers', 'harass', 'harassed', 'harassing', 'harassment', 'hardship', 'hardships', 'harm',
                       'harmed', 'harmful', 'harmfully', 'harming', 'harms', 'harsh', 'harsher', 'harshest', 'harshly',
                       'harshness', 'hazard', 'hazardous', 'hazards', 'hinder', 'hindered', 'hindering', 'hinders',
                       'hindrance', 'hindrances', 'hostile', 'hostility', 'hurt', 'hurting', 'idle', 'idled', 'idling',
                       'ignore', 'ignored', 'ignores', 'ignoring', 'ill', 'illegal', 'illegalities', 'illegality',
                       'illegally', 'illegible', 'illicit', 'illicitly', 'illiquid', 'illiquidity', 'imbalance',
                       'imbalances', 'immature', 'immoral', 'impair', 'impaired', 'impairing', 'impairment',
                       'impairments', 'impairs', 'impasse', 'impasses', 'impede', 'impeded', 'impedes', 'impediment',
                       'impediments', 'impeding', 'impending', 'imperative', 'imperfection', 'imperfections', 'imperil',
                       'impermissible', 'implicate', 'implicated', 'implicates', 'implicating', 'impossibility',
                       'impossible', 'impound', 'impounded', 'impounding', 'impounds', 'impracticable', 'impractical',
                       'impracticalities', 'impracticality', 'imprisonment', 'improper', 'improperly', 'improprieties',
                       'impropriety', 'imprudent', 'imprudently', 'inability', 'inaccessible', 'inaccuracies',
                       'inaccuracy', 'inaccurate', 'inaccurately', 'inaction', 'inactions', 'inactivate', 'inactivated',
                       'inactivates', 'inactivating', 'inactivation', 'inactivations', 'inactivity', 'inadequacies',
                       'inadequacy', 'inadequate', 'inadequately', 'inadvertent', 'inadvertently', 'inadvisability',
                       'inadvisable', 'inappropriate', 'inappropriately', 'inattention', 'incapable', 'incapacitated',
                       'incapacity', 'incarcerate', 'incarcerated', 'incarcerates', 'incarcerating', 'incarceration',
                       'incarcerations', 'incidence', 'incidences', 'incident', 'incidents', 'incompatibilities',
                       'incompatibility', 'incompatible', 'incompetence', 'incompetency', 'incompetent',
                       'incompetently', 'incompetents', 'incomplete', 'incompletely', 'incompleteness', 'inconclusive',
                       'inconsistencies', 'inconsistency', 'inconsistent', 'inconsistently', 'inconvenience',
                       'inconveniences', 'inconvenient', 'incorrect', 'incorrectly', 'incorrectness', 'indecency',
                       'indecent', 'indefeasible', 'indefeasibly', 'indict', 'indictable', 'indicted', 'indicting',
                       'indictment', 'indictments', 'ineffective', 'ineffectively', 'ineffectiveness', 'inefficiencies',
                       'inefficiency', 'inefficient', 'inefficiently', 'ineligibility', 'ineligible', 'inequitable',
                       'inequitably', 'inequities', 'inequity', 'inevitable', 'inexperience', 'inexperienced',
                       'inferior', 'inflicted', 'infraction', 'infractions', 'infringe', 'infringed', 'infringement',
                       'infringements', 'infringes', 'infringing', 'inhibited', 'inimical', 'injunction', 'injunctions',
                       'injure', 'injured', 'injures', 'injuries', 'injuring', 'injurious', 'injury', 'inordinate',
                       'inordinately', 'inquiry', 'insecure', 'insensitive', 'insolvencies', 'insolvency', 'insolvent',
                       'instability', 'insubordination', 'insufficiency', 'insufficient', 'insufficiently',
                       'insurrection', 'insurrections', 'intentional', 'interfere', 'interfered', 'interference',
                       'interferences', 'interferes', 'interfering', 'intermittent', 'intermittently', 'interrupt',
                       'interrupted', 'interrupting', 'interruption', 'interruptions', 'interrupts', 'intimidation',
                       'intrusion', 'invalid', 'invalidate', 'invalidated', 'invalidates', 'invalidating',
                       'invalidation', 'invalidity', 'investigate', 'investigated', 'investigates', 'investigating',
                       'investigation', 'investigations', 'involuntarily', 'involuntary', 'irreconcilable',
                       'irreconcilably', 'irrecoverable', 'irrecoverably', 'irregular', 'irregularities',
                       'irregularity', 'irregularly', 'irreparable', 'irreparably', 'irreversible', 'jeopardize',
                       'jeopardized', 'justifiable', 'kickback', 'kickbacks', 'knowingly', 'lack', 'lacked', 'lacking',
                       'lackluster', 'lacks', 'lag', 'lagged', 'lagging', 'lags', 'lapse', 'lapsed', 'lapses',
                       'lapsing', 'late', 'laundering', 'layoff', 'layoffs', 'lie', 'limitation', 'limitations',
                       'lingering', 'liquidate', 'liquidated', 'liquidates', 'liquidating', 'liquidation',
                       'liquidations', 'liquidator', 'liquidators', 'litigant', 'litigants', 'litigate', 'litigated',
                       'litigates', 'litigating', 'litigation', 'litigations', 'lockout', 'lockouts', 'lose', 'loses',
                       'losing', 'loss', 'losses', 'lost', 'lying', 'malfeasance', 'malfunction', 'malfunctioned',
                       'malfunctioning', 'malfunctions', 'malice', 'malicious', 'maliciously', 'malpractice',
                       'manipulate', 'manipulated', 'manipulates', 'manipulating', 'manipulation', 'manipulations',
                       'manipulative', 'markdown', 'markdowns', 'misapplication', 'misapplications', 'misapplied',
                       'misapplies', 'misapply', 'misapplying', 'misappropriate', 'misappropriated', 'misappropriates',
                       'misappropriating', 'misappropriation', 'misappropriations', 'misbranded', 'miscalculate',
                       'miscalculated', 'miscalculates', 'miscalculating', 'miscalculation', 'miscalculations',
                       'mischaracterization', 'mischief', 'misclassification', 'misclassifications', 'misclassified',
                       'misclassify', 'miscommunication', 'misconduct', 'misdated', 'misdemeanor', 'misdemeanors',
                       'misdirected', 'mishandle', 'mishandled', 'mishandles', 'mishandling', 'misinform',
                       'misinformation', 'misinformed', 'misinforming', 'misinforms', 'misinterpret',
                       'misinterpretation', 'misinterpretations', 'misinterpreted', 'misinterpreting', 'misinterprets',
                       'misjudge', 'misjudged', 'misjudges', 'misjudging', 'misjudgment', 'misjudgments', 'mislabel',
                       'mislabeled', 'mislabeling', 'mislabelled', 'mislabels', 'mislead', 'misleading', 'misleadingly',
                       'misleads', 'misled', 'mismanage', 'mismanaged', 'mismanagement', 'mismanages', 'mismanaging',
                       'mismatch', 'mismatched', 'mismatches', 'mismatching', 'misplaced', 'misprice', 'mispricing',
                       'mispricings', 'misrepresent', 'misrepresentation', 'misrepresentations', 'misrepresented',
                       'misrepresenting', 'misrepresents', 'miss', 'missed', 'misses', 'misstate', 'misstated',
                       'misstatement', 'misstatements', 'misstates', 'misstating', 'misstep', 'missteps', 'mistake',
                       'mistaken', 'mistakenly', 'mistakes', 'mistaking', 'mistrial', 'mistrials', 'misunderstand',
                       'misunderstanding', 'misunderstandings', 'misunderstood', 'misuse', 'misused', 'misuses',
                       'misusing', 'monopolistic', 'monopolists', 'monopolization', 'monopolize', 'monopolized',
                       'monopolizes', 'monopolizing', 'monopoly', 'moratoria', 'moratorium', 'moratoriums',
                       'mothballed', 'mothballing', 'negative', 'negatively', 'negatives', 'neglect', 'neglected',
                       'neglectful', 'neglecting', 'neglects', 'negligence', 'negligences', 'negligent', 'negligently',
                       'nonattainment', 'noncompetitive', 'noncompliance', 'noncompliances', 'noncompliant',
                       'noncomplying', 'nonconforming', 'nonconformities', 'nonconformity', 'nondisclosure',
                       'nonfunctional', 'nonpayment', 'nonpayments', 'nonperformance', 'nonperformances',
                       'nonperforming', 'nonproducing', 'nonproductive', 'nonrecoverable', 'nonrenewal', 'nuisance',
                       'nuisances', 'nullification', 'nullifications', 'nullified', 'nullifies', 'nullify',
                       'nullifying', 'objected', 'objecting', 'objection', 'objectionable', 'objectionably',
                       'objections', 'obscene', 'obscenity', 'obsolescence', 'obsolete', 'obstacle', 'obstacles',
                       'obstruct', 'obstructed', 'obstructing', 'obstruction', 'obstructions', 'offence', 'offences',
                       'offend', 'offended', 'offender', 'offenders', 'offending', 'offends', 'omission', 'omissions',
                       'omit', 'omits', 'omitted', 'omitting', 'onerous', 'opportunistic', 'opportunistically',
                       'oppose', 'opposed', 'opposes', 'opposing', 'opposition', 'oppositions', 'outage', 'outages',
                       'outdated', 'outmoded', 'overage', 'overages', 'overbuild', 'overbuilding', 'overbuilds',
                       'overbuilt', 'overburden', 'overburdened', 'overburdening', 'overcapacities', 'overcapacity',
                       'overcharge', 'overcharged', 'overcharges', 'overcharging', 'overcome', 'overcomes',
                       'overcoming', 'overdue', 'overestimate', 'overestimated', 'overestimates', 'overestimating',
                       'overestimation', 'overestimations', 'overload', 'overloaded', 'overloading', 'overloads',
                       'overlook', 'overlooked', 'overlooking', 'overlooks', 'overpaid', 'overpayment', 'overpayments',
                       'overproduced', 'overproduces', 'overproducing', 'overproduction', 'overrun', 'overrunning',
                       'overruns', 'overshadow', 'overshadowed', 'overshadowing', 'overshadows', 'overstate',
                       'overstated', 'overstatement', 'overstatements', 'overstates', 'overstating', 'oversupplied',
                       'oversupplies', 'oversupply', 'oversupplying', 'overtly', 'overturn', 'overturned',
                       'overturning', 'overturns', 'overvalue', 'overvalued', 'overvaluing', 'panic', 'panics',
                       'penalize', 'penalized', 'penalizes', 'penalizing', 'penalties', 'penalty', 'peril', 'perils',
                       'perjury', 'perpetrate', 'perpetrated', 'perpetrates', 'perpetrating', 'perpetration', 'persist',
                       'persisted', 'persistence', 'persistent', 'persistently', 'persisting', 'persists', 'pervasive',
                       'pervasively', 'pervasiveness', 'petty', 'picket', 'picketed', 'picketing', 'plaintiff',
                       'plaintiffs', 'plea', 'plead', 'pleaded', 'pleading', 'pleadings', 'pleads', 'pleas', 'pled',
                       'poor', 'poorly', 'poses', 'posing', 'postpone', 'postponed', 'postponement', 'postponements',
                       'postpones', 'postponing', 'precipitated', 'precipitous', 'precipitously', 'preclude',
                       'precluded', 'precludes', 'precluding', 'predatory', 'prejudice', 'prejudiced', 'prejudices',
                       'prejudicial', 'prejudicing', 'premature', 'prematurely', 'pressing', 'pretrial', 'preventing',
                       'prevention', 'prevents', 'problem', 'problematic', 'problematical', 'problems', 'prolong',
                       'prolongation', 'prolongations', 'prolonged', 'prolonging', 'prolongs', 'prone', 'prosecute',
                       'prosecuted', 'prosecutes', 'prosecuting', 'prosecution', 'prosecutions', 'protest', 'protested',
                       'protester', 'protesters', 'protesting', 'protestor', 'protestors', 'protests', 'protracted',
                       'protraction', 'provoke', 'provoked', 'provokes', 'provoking', 'punished', 'punishes',
                       'punishing', 'punishment', 'punishments', 'punitive', 'purport', 'purported', 'purportedly',
                       'purporting', 'purports', 'question', 'questionable', 'questionably', 'questioned',
                       'questioning', 'questions', 'quit', 'quitting', 'racketeer', 'racketeering', 'rationalization',
                       'rationalizations', 'rationalize', 'rationalized', 'rationalizes', 'rationalizing',
                       'reassessment', 'reassessments', 'reassign', 'reassigned', 'reassigning', 'reassignment',
                       'reassignments', 'reassigns', 'recall', 'recalled', 'recalling', 'recalls', 'recession',
                       'recessionary', 'recessions', 'reckless', 'recklessly', 'recklessness', 'redact', 'redacted',
                       'redacting', 'redaction', 'redactions', 'redefault', 'redefaulted', 'redefaults', 'redress',
                       'redressed', 'redresses', 'redressing', 'refusal', 'refusals', 'refuse', 'refused', 'refuses',
                       'refusing', 'reject', 'rejected', 'rejecting', 'rejection', 'rejections', 'rejects',
                       'relinquish', 'relinquished', 'relinquishes', 'relinquishing', 'relinquishment',
                       'relinquishments', 'reluctance', 'reluctant', 'renegotiate', 'renegotiated', 'renegotiates',
                       'renegotiating', 'renegotiation', 'renegotiations', 'renounce', 'renounced', 'renouncement',
                       'renouncements', 'renounces', 'renouncing', 'reparation', 'reparations', 'repossessed',
                       'repossesses', 'repossessing', 'repossession', 'repossessions', 'repudiate', 'repudiated',
                       'repudiates', 'repudiating', 'repudiation', 'repudiations', 'resign', 'resignation',
                       'resignations', 'resigned', 'resigning', 'resigns', 'restate', 'restated', 'restatement',
                       'restatements', 'restates', 'restating', 'restructure', 'restructured', 'restructures',
                       'restructuring', 'restructurings', 'retaliate', 'retaliated', 'retaliates', 'retaliating',
                       'retaliation', 'retaliations', 'retaliatory', 'retribution', 'retributions', 'revocation',
                       'revocations', 'revoke', 'revoked', 'revokes', 'revoking', 'ridicule', 'ridiculed', 'ridicules',
                       'ridiculing', 'riskier', 'riskiest', 'risky', 'sabotage', 'sacrifice', 'sacrificed',
                       'sacrifices', 'sacrificial', 'sacrificing', 'scandalous', 'scandals', 'scrutinize',
                       'scrutinized', 'scrutinizes', 'scrutinizing', 'scrutiny', 'secrecy', 'seize', 'seized', 'seizes',
                       'seizing', 'sentenced', 'sentencing', 'serious', 'seriously', 'seriousness', 'setback',
                       'setbacks', 'sever', 'severe', 'severed', 'severely', 'severities', 'severity', 'sharply',
                       'shocked', 'shortage', 'shortages', 'shortfall', 'shortfalls', 'shrinkage', 'shrinkages', 'shut',
                       'shutdown', 'shutdowns', 'shuts', 'shutting', 'slander', 'slandered', 'slanderous', 'slanders',
                       'slippage', 'slippages', 'slow', 'slowdown', 'slowdowns', 'slowed', 'slower', 'slowest',
                       'slowing', 'slowly', 'slowness', 'sluggish', 'sluggishly', 'sluggishness', 'solvencies',
                       'solvency', 'spam', 'spammers', 'spamming', 'staggering', 'stagnant', 'stagnate', 'stagnated',
                       'stagnates', 'stagnating', 'stagnation', 'standstill', 'standstills', 'stolen', 'stoppage',
                       'stoppages', 'stopped', 'stopping', 'stops', 'strain', 'strained', 'straining', 'strains',
                       'stress', 'stressed', 'stresses', 'stressful', 'stressing', 'stringent', 'subjected',
                       'subjecting', 'subjection', 'subpoena', 'subpoenaed', 'subpoenas', 'substandard', 'sue', 'sued',
                       'sues', 'suffer', 'suffered', 'suffering', 'suffers', 'suing', 'summoned', 'summoning',
                       'summons', 'summonses', 'susceptibility', 'susceptible', 'suspect', 'suspected', 'suspects',
                       'suspend', 'suspended', 'suspending', 'suspends', 'suspension', 'suspensions', 'suspicion',
                       'suspicions', 'suspicious', 'suspiciously', 'taint', 'tainted', 'tainting', 'taints', 'tampered',
                       'tense', 'terminate', 'terminated', 'terminates', 'terminating', 'termination', 'terminations',
                       'testify', 'testifying', 'threat', 'threaten', 'threatened', 'threatening', 'threatens',
                       'threats', 'tightening', 'tolerate', 'tolerated', 'tolerates', 'tolerating', 'toleration',
                       'tortuous', 'tortuously', 'tragedies', 'tragedy', 'tragic', 'tragically', 'traumatic', 'trouble',
                       'troubled', 'troubles', 'turbulence', 'turmoil', 'unable', 'unacceptable', 'unacceptably',
                       'unaccounted', 'unannounced', 'unanticipated', 'unapproved', 'unattractive', 'unauthorized',
                       'unavailability', 'unavailable', 'unavoidable', 'unavoidably', 'unaware', 'uncollectable',
                       'uncollected', 'uncollectibility', 'uncollectible', 'uncollectibles', 'uncompetitive',
                       'uncompleted', 'unconscionable', 'unconscionably', 'uncontrollable', 'uncontrollably',
                       'uncontrolled', 'uncorrected', 'uncover', 'uncovered', 'uncovering', 'uncovers', 'undeliverable',
                       'undelivered', 'undercapitalized', 'undercut', 'undercuts', 'undercutting', 'underestimate',
                       'underestimated', 'underestimates', 'underestimating', 'underestimation', 'underfunded',
                       'underinsured', 'undermine', 'undermined', 'undermines', 'undermining', 'underpaid',
                       'underpayment', 'underpayments', 'underpays', 'underperform', 'underperformance',
                       'underperformed', 'underperforming', 'underperforms', 'underproduced', 'underproduction',
                       'underreporting', 'understate', 'understated', 'understatement', 'understatements',
                       'understates', 'understating', 'underutilization', 'underutilized', 'undesirable', 'undesired',
                       'undetected', 'undetermined', 'undisclosed', 'undocumented', 'undue', 'unduly', 'uneconomic',
                       'uneconomical', 'uneconomically', 'unemployed', 'unemployment', 'unethical', 'unethically',
                       'unexcused', 'unexpected', 'unexpectedly', 'unfair', 'unfairly', 'unfavorability', 'unfavorable',
                       'unfavorably', 'unfavourable', 'unfeasible', 'unfit', 'unfitness', 'unforeseeable', 'unforeseen',
                       'unforseen', 'unfortunate', 'unfortunately', 'unfounded', 'unfriendly', 'unfulfilled',
                       'unfunded', 'uninsured', 'unintended', 'unintentional', 'unintentionally', 'unjust',
                       'unjustifiable', 'unjustifiably', 'unjustified', 'unjustly', 'unknowing', 'unknowingly',
                       'unlawful', 'unlawfully', 'unlicensed', 'unliquidated', 'unmarketable', 'unmerchantable',
                       'unmeritorious', 'unnecessarily', 'unnecessary', 'unneeded', 'unobtainable', 'unoccupied',
                       'unpaid', 'unperformed', 'unplanned', 'unpopular', 'unpredictability', 'unpredictable',
                       'unpredictably', 'unpredicted', 'unproductive', 'unprofitability', 'unprofitable', 'unqualified',
                       'unrealistic', 'unreasonable', 'unreasonableness', 'unreasonably', 'unreceptive',
                       'unrecoverable', 'unrecovered', 'unreimbursed', 'unreliable', 'unremedied', 'unreported',
                       'unresolved', 'unrest', 'unsafe', 'unsalable', 'unsaleable', 'unsatisfactory', 'unsatisfied',
                       'unsavory', 'unscheduled', 'unsellable', 'unsold', 'unsound', 'unstabilized', 'unstable',
                       'unsubstantiated', 'unsuccessful', 'unsuccessfully', 'unsuitability', 'unsuitable', 'unsuitably',
                       'unsuited', 'unsure', 'unsuspected', 'unsuspecting', 'unsustainable', 'untenable', 'untimely',
                       'untrusted', 'untruth', 'untruthful', 'untruthfully', 'untruthfulness', 'untruths', 'unusable',
                       'unwanted', 'unwarranted', 'unwelcome', 'unwilling', 'unwillingness', 'upset', 'urgency',
                       'urgent', 'usurious', 'usurp', 'usurped', 'usurping', 'usurps', 'usury', 'vandalism', 'verdict',
                       'verdicts', 'vetoed', 'victims', 'violate', 'violated', 'violates', 'violating', 'violation',
                       'violations', 'violative', 'violator', 'violators', 'violence', 'violent', 'violently',
                       'vitiate', 'vitiated', 'vitiates', 'vitiating', 'vitiation', 'voided', 'voiding', 'volatile',
                       'volatility', 'vulnerabilities', 'vulnerability', 'vulnerable', 'vulnerably', 'warn', 'warned',
                       'warning', 'warnings', 'warns', 'wasted', 'wasteful', 'wasting', 'weak', 'weaken', 'weakened',
                       'weakening', 'weakens', 'weaker', 'weakest', 'weakly', 'weakness', 'weaknesses', 'willfully',
                       'worries', 'worry', 'worrying', 'worse', 'worsen', 'worsened', 'worsening', 'worsens', 'worst',
                       'worthless', 'writedown', 'writedowns', 'writeoff', 'writeoffs', 'wrong', 'wrongdoing',
                       'wrongdoings', 'wrongful', 'wrongfully', 'wrongly'],
          'Positive': ['able', 'abundance', 'abundant', 'acclaimed', 'accomplish', 'accomplished', 'accomplishes',
                       'accomplishing', 'accomplishment', 'accomplishments', 'achieve', 'achieved', 'achievement',
                       'achievements', 'achieves', 'achieving', 'adequately', 'advancement', 'advancements', 'advances',
                       'advancing', 'advantage', 'advantaged', 'advantageous', 'advantageously', 'advantages',
                       'alliance', 'alliances', 'assure', 'assured', 'assures', 'assuring', 'attain', 'attained',
                       'attaining', 'attainment', 'attainments', 'attains', 'attractive', 'attractiveness', 'beautiful',
                       'beautifully', 'beneficial', 'beneficially', 'benefit', 'benefited', 'benefiting', 'benefitted',
                       'benefitting', 'best', 'better', 'bolstered', 'bolstering', 'bolsters', 'boom', 'booming',
                       'boost', 'boosted', 'breakthrough', 'breakthroughs', 'brilliant', 'charitable', 'collaborate',
                       'collaborated', 'collaborates', 'collaborating', 'collaboration', 'collaborations',
                       'collaborative', 'collaborator', 'collaborators', 'compliment', 'complimentary', 'complimented',
                       'complimenting', 'compliments', 'conclusive', 'conclusively', 'conducive', 'confident',
                       'constructive', 'constructively', 'courteous', 'creative', 'creatively', 'creativeness',
                       'creativity', 'delight', 'delighted', 'delightful', 'delightfully', 'delighting', 'delights',
                       'dependability', 'dependable', 'desirable', 'desired', 'despite', 'destined', 'diligent',
                       'diligently', 'distinction', 'distinctions', 'distinctive', 'distinctively', 'distinctiveness',
                       'dream', 'easier', 'easily', 'easy', 'effective', 'efficiencies', 'efficiency', 'efficient',
                       'efficiently', 'empower', 'empowered', 'empowering', 'empowers', 'enable', 'enabled', 'enables',
                       'enabling', 'encouraged', 'encouragement', 'encourages', 'encouraging', 'enhance', 'enhanced',
                       'enhancement', 'enhancements', 'enhances', 'enhancing', 'enjoy', 'enjoyable', 'enjoyably',
                       'enjoyed', 'enjoying', 'enjoyment', 'enjoys', 'enthusiasm', 'enthusiastic', 'enthusiastically',
                       'excellence', 'excellent', 'excelling', 'excels', 'exceptional', 'exceptionally', 'excited',
                       'excitement', 'exciting', 'exclusive', 'exclusively', 'exclusiveness', 'exclusives',
                       'exclusivity', 'exemplary', 'fantastic', 'favorable', 'favorably', 'favored', 'favoring',
                       'favorite', 'favorites', 'friendly', 'gain', 'gained', 'gaining', 'gains', 'good', 'great',
                       'greater', 'greatest', 'greatly', 'greatness', 'happiest', 'happily', 'happiness', 'happy',
                       'highest', 'honor', 'honorable', 'honored', 'honoring', 'honors', 'ideal', 'impress',
                       'impressed', 'impresses', 'impressing', 'impressive', 'impressively', 'improve', 'improved',
                       'improvement', 'improvements', 'improves', 'improving', 'incredible', 'incredibly',
                       'influential', 'informative', 'ingenuity', 'innovate', 'innovated', 'innovates', 'innovating',
                       'innovation', 'innovations', 'innovative', 'innovativeness', 'innovator', 'innovators',
                       'insightful', 'inspiration', 'inspirational', 'integrity', 'invent', 'invented', 'inventing',
                       'invention', 'inventions', 'inventive', 'inventiveness', 'inventor', 'inventors', 'leadership',
                       'leading', 'loyal', 'lucrative', 'meritorious', 'opportunities', 'opportunity', 'optimistic',
                       'outperform', 'outperformed', 'outperforming', 'outperforms', 'perfect', 'perfected',
                       'perfectly', 'perfects', 'pleasant', 'pleasantly', 'pleased', 'pleasure', 'plentiful', 'popular',
                       'popularity', 'positive', 'positively', 'preeminence', 'preeminent', 'premier', 'premiere',
                       'prestige', 'prestigious', 'proactive', 'proactively', 'proficiency', 'proficient',
                       'proficiently', 'profitability', 'profitable', 'profitably', 'progress', 'progressed',
                       'progresses', 'progressing', 'prospered', 'prospering', 'prosperity', 'prosperous', 'prospers',
                       'rebound', 'rebounded', 'rebounding', 'receptive', 'regain', 'regained', 'regaining', 'resolve',
                       'revolutionize', 'revolutionized', 'revolutionizes', 'revolutionizing', 'reward', 'rewarded',
                       'rewarding', 'rewards', 'satisfaction', 'satisfactorily', 'satisfactory', 'satisfied',
                       'satisfies', 'satisfy', 'satisfying', 'smooth', 'smoothing', 'smoothly', 'smooths', 'solves',
                       'solving', 'spectacular', 'spectacularly', 'stability', 'stabilization', 'stabilizations',
                       'stabilize', 'stabilized', 'stabilizes', 'stabilizing', 'stable', 'strength', 'strengthen',
                       'strengthened', 'strengthening', 'strengthens', 'strengths', 'strong', 'stronger', 'strongest',
                       'succeed', 'succeeded', 'succeeding', 'succeeds', 'success', 'successes', 'successful',
                       'successfully', 'superior', 'surpass', 'surpassed', 'surpasses', 'surpassing', 'transparency',
                       'tremendous', 'tremendously', 'unmatched', 'unparalleled', 'unsurpassed', 'upturn', 'upturns',
                       'valuable', 'versatile', 'versatility', 'vibrancy', 'vibrant', 'win', 'winner', 'winners',
                       'winning', 'worthy']}

## Positive Score

In [31]:
# Calculating positive score

def positive_score(text):
    PosWords = 0
    
    for word in text:
        if word in lmdict['Positive']:
            PosWords  += 1
    
    Pos = PosWords
    return Pos

## Negative Score

In [32]:
# Calculating Negative score

def negative_score(text):
    NegWords=0
    
    for word in text:
        if word in lmdict['Negative']:
            NegWords -=1
    Neg = NegWords 
    Neg = Neg * -1
    return Neg

## Polarity Score

In [33]:
# Calculating polarity score

def polarity_score(positiveScore, negativeScore):
    pol_score = (positiveScore - negativeScore) / ((positiveScore + negativeScore) + 0.000001)
    return pol_score

## Subjectivity Score

In [34]:
# Calculating Subjectivity Score

def subjectivity_score(positiveScore, negativeScore, length_stopword_removed_content):
    sub_score = (positiveScore + negativeScore) / ((length_stopword_removed_content) + 0.000001)
    return sub_score

## Average Sentence Length

In [35]:
# Calculating Average sentence length 
# Average Sentence Length = the number of words / the number of sentences
     
def average_sentence_length(val):
    sentence_list = sent_tokenize(val)
    tokens = tokenizer(val)
    totalWordCount = len(tokens)
    totalSentences = len(sentence_list)
    average_sent = 0
    if totalSentences != 0:
        average_sent = totalWordCount / totalSentences
    
    average_sent_length= average_sent
    
    return round(average_sent_length)

## Percentage of Complex Word 

In [36]:
# Calculating percentage of complex word 
# Percentage of Complex words = the number of complex words / the number of words 

def percentage_complex_word(text):
    tokens = tokenizer(text)
    complexWord = 0
    complex_word_percentage = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    if len(tokens) != 0:
        complex_word_percentage = complexWord/len(tokens)
    
    return complex_word_percentage

## Fog Index

In [37]:
# calculating Fog Index 
# Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

def fog_index(averageSentenceLength, percentageComplexWord):
    fogIndex = 0.4 * (averageSentenceLength + percentageComplexWord)
    return fogIndex

## Average Number of Words Per Sentence

In [38]:
# Average Number of Words Per Sentence 
# Average Number of Words Per Sentence = the total number of words / the total number of sentences
     
def average_num_of_words_per_sentence(val):
    sentence_list = sent_tokenize(val)
    tokens = val.split()
    totalWordCount = len(tokens)
    totalSentences = len(sentence_list)
    average_sent = 0
    if totalSentences != 0:
        avg_num_words = totalWordCount / totalSentences
    
    average_number_words= avg_num_words
    
    return round(average_number_words)

## Complex Words

In [39]:
# Counting complex words

def complex_word_count(text):
    tokens = tokenizer(text)
    complexWord = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    return complexWord

## Total Words

In [40]:
#Counting total words

def total_word_count(text):
    tokens = tokenizer(text)
    return len(tokens)

## Syllable Count Per Word

In [41]:
# Syllable Count Per Word

def Syllable_Count_Per_Word(text):
    
    string = ""
    for word in text.split():
        if word[-2:]== "es" or word[-2:]== "ed":
            pass
        else:
            string= string +" "+ word

    vowels=0
    for i in string.replace(" ",""):
        if(i=='a' or i=='e' or i=='i' or i=='o' or i=='u' or i=='A' or i=='E' or i=='I' or i=='O' or i=='U'):
            vowels=vowels+1
    
    return vowels

## Personal Pronouns

In [42]:
# Personal Pronouns

def count_pronoun(string):

    string= string.lower()
    words = string.split()

    count = 0
    
    for i in words:
        if(i=='i' or i=='we' or i=='my' or i=='ours' or i=='us' or i=='I' or i=='We' or i=='My' or i=='Ours' or i=='Us'):
            count += 1
    return count

## Average Word Length

In [108]:
# avg world length

def avg_word_length(text):
    b=text.replace(" ","")
    c= text.split()
    Sum_of_the_total_number_of_characters = len(b)
    Total_number_of_words = len(c)
    average_word_length = Sum_of_the_total_number_of_characters / Total_number_of_words
    
    return average_word_length

## Applying Formula

In [44]:
# applying above function and formula

df["POSITIVE SCORE"]= df['stopword_removed_content'].apply(positive_score)
df["NEGATIVE SCORE"]= df['stopword_removed_content'].apply(negative_score)
df['POLARITY SCORE'] = np.vectorize(polarity_score)(df['POSITIVE SCORE'],df['NEGATIVE SCORE'])
df['SUBJECTIVITY SCORE'] = np.vectorize(subjectivity_score)(df['POSITIVE SCORE'],df['NEGATIVE SCORE'],df['length_stopword_removed_content'])
df['AVG SENTENCE LENGTH'] = df["Content String"].apply(average_sentence_length)
df['PERCENTAGE OF COMPLEX WORDS'] = df["Content String"].apply(percentage_complex_word)
df['FOG INDEX'] = np.vectorize(fog_index)(df['AVG SENTENCE LENGTH'],df['PERCENTAGE OF COMPLEX WORDS'])
df['AVG NUMBER OF WORDS PER SENTENCE'] = df["Content String"].apply(average_sentence_length)
df['COMPLEX WORD COUNT']= df["Content String"].apply(complex_word_count)
df['WORD COUNT'] = df["Content String"].apply(total_word_count)
df['SYLLABLE PER WORD'] = df["Content String"].apply(Syllable_Count_Per_Word)
df["PERSONAL PRONOUNS"]= df["Content String"].apply(count_pronoun)
df["AVG WORD LENGTH"]= df["Content String"].apply(avg_word_length)

In [67]:
df

,URL_ID,URL,Title,Content,Content String,Length Content String,stopword_removed_content,length_stopword_removed_content,POSITIVE SCORE,NEGATIVE SCORE,...,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,1,https://insights.blackcoffer.com/how-is-login-...,How is Login Logout Time Tracking for Employee...,[When people hear AI they often think about se...,When people hear AI they often think about sen...,4286,"[people, hear, sentient, robots, magic, boxes,...",327,2,6,...,0.024465,16,0.388379,6.555352,16,127,327,1239,4,5.234352
1,2,https://insights.blackcoffer.com/how-does-ai-h...,How does AI help to monitor Retail Shelf watch...,[With increasing computing power and more data...,"With increasing computing power and more data,...",3984,"[increasing, computing, data, potential, algor...",303,8,6,...,0.046205,15,0.445545,6.178218,15,135,303,1155,2,5.479675
2,3,https://insights.blackcoffer.com/ai-and-its-im...,AI and its impact on the Fashion Industry - Bl...,[If you were a fan of the 90’s film Clueless b...,If you were a fan of the ’s film Clueless ba...,11324,"[fan, film, clueless, ll, remember, protagonis...",863,31,23,...,0.062572,22,0.483198,8.993279,22,417,863,3231,10,5.619883
3,4,https://insights.blackcoffer.com/how-do-deep-l...,How do deep learning models predict old and ne...,"[Understanding exactly how data is ingested, a...","Understanding exactly how data is ingested, an...",2706,"[understanding, data, ingested, analyzed, retu...",213,3,1,...,0.018779,15,0.502347,6.200939,15,107,213,818,1,5.384434
4,5,https://insights.blackcoffer.com/how-artificia...,How artificial intelligence can boost your pro...,"[From the stone age to the modern world, from ...","From the stone age to the modern world, from h...",7484,"[age, modern, world, hunting, gathering, culti...",527,18,17,...,0.066414,11,0.483871,4.593548,11,255,527,2304,26,5.254788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,167,https://insights.blackcoffer.com/role-big-data...,Role of Big Data in Academia - Blackcoffer Ins...,"[Can academia, researchers, decision makers an...","Can academia, researchers, decision makers and...",9235,"[academia, researchers, decision, makers, poli...",652,15,37,...,0.079755,14,0.446319,5.778528,14,291,652,2740,14,5.300613
166,168,https://insights.blackcoffer.com/sales-forecas...,Methods for Sales Forecasting in Retail - Blac...,[Inventory planning is a fundamental part of r...,Inventory planning is a fundamental part of re...,6132,"[inventory, planning, fundamental, part, retai...",480,18,15,...,0.068750,17,0.477083,6.990833,17,229,480,1745,0,5.722588
167,169,https://insights.blackcoffer.com/detect-data-e...,Detect Data ExFiltration over the Network - Bl...,[Insider threat detection specifically to dete...,Insider threat detection specifically to detec...,6222,"[insider, threat, detection, specifically, det...",520,4,47,...,0.098077,12,0.446154,4.978462,12,232,520,1829,6,5.688242
168,170,https://insights.blackcoffer.com/data-exfiltra...,What is Data Exfiltration? - Blackcoffer Insights,"[If we talk in terms of our general life, Exfi...","If we talk in terms of our general life, Exfil...",2989,"[talk, terms, general, life, exfiltrate, surre...",205,4,8,...,0.058537,10,0.487805,4.195122,10,100,205,900,11,5.118609


In [68]:
df1 = df.iloc[:,[0,1,8,9,10,11,12,13,14,15,16,17,18,19,20]]

## Result Dataframe

In [69]:
df1

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,1,https://insights.blackcoffer.com/how-is-login-...,2,6,-0.500000,0.024465,16,0.388379,6.555352,16,127,327,1239,4,5.234352
1,2,https://insights.blackcoffer.com/how-does-ai-h...,8,6,0.142857,0.046205,15,0.445545,6.178218,15,135,303,1155,2,5.479675
2,3,https://insights.blackcoffer.com/ai-and-its-im...,31,23,0.148148,0.062572,22,0.483198,8.993279,22,417,863,3231,10,5.619883
3,4,https://insights.blackcoffer.com/how-do-deep-l...,3,1,0.500000,0.018779,15,0.502347,6.200939,15,107,213,818,1,5.384434
4,5,https://insights.blackcoffer.com/how-artificia...,18,17,0.028571,0.066414,11,0.483871,4.593548,11,255,527,2304,26,5.254788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,167,https://insights.blackcoffer.com/role-big-data...,15,37,-0.423077,0.079755,14,0.446319,5.778528,14,291,652,2740,14,5.300613
166,168,https://insights.blackcoffer.com/sales-forecas...,18,15,0.090909,0.068750,17,0.477083,6.990833,17,229,480,1745,0,5.722588
167,169,https://insights.blackcoffer.com/detect-data-e...,4,47,-0.843137,0.098077,12,0.446154,4.978462,12,232,520,1829,6,5.688242
168,170,https://insights.blackcoffer.com/data-exfiltra...,4,8,-0.333333,0.058537,10,0.487805,4.195122,10,100,205,900,11,5.118609


In [70]:
df2 = df.iloc[:,[0,1,2,4]]

In [71]:
df2

,URL_ID,URL,Title,Content String
0,1,https://insights.blackcoffer.com/how-is-login-...,How is Login Logout Time Tracking for Employee...,When people hear AI they often think about sen...
1,2,https://insights.blackcoffer.com/how-does-ai-h...,How does AI help to monitor Retail Shelf watch...,"With increasing computing power and more data,..."
2,3,https://insights.blackcoffer.com/ai-and-its-im...,AI and its impact on the Fashion Industry - Bl...,If you were a fan of the ’s film Clueless ba...
3,4,https://insights.blackcoffer.com/how-do-deep-l...,How do deep learning models predict old and ne...,"Understanding exactly how data is ingested, an..."
4,5,https://insights.blackcoffer.com/how-artificia...,How artificial intelligence can boost your pro...,"From the stone age to the modern world, from h..."
...,...,...,...,...
165,167,https://insights.blackcoffer.com/role-big-data...,Role of Big Data in Academia - Blackcoffer Ins...,"Can academia, researchers, decision makers and..."
166,168,https://insights.blackcoffer.com/sales-forecas...,Methods for Sales Forecasting in Retail - Blac...,Inventory planning is a fundamental part of re...
167,169,https://insights.blackcoffer.com/detect-data-e...,Detect Data ExFiltration over the Network - Bl...,Insider threat detection specifically to detec...
168,170,https://insights.blackcoffer.com/data-exfiltra...,What is Data Exfiltration? - Blackcoffer Insights,"If we talk in terms of our general life, Exfil..."


## Writing to Text Files

In [102]:
# Writing to text files
a= df2["URL_ID"]
b= df2["Title"]
c= df2["Content String"]

for i in range(len(a)):
    text_file = open(str(a[i])+".txt", "w",encoding="utf-8")
    text_file.write(str(b[i])+"\n"+str(c[i]))
    text_file.close()

## Writing to a CSV File

In [107]:
# Writing to csv file
df1.to_csv('BlackcofferTextAnalysisOutput.csv', sep=',', encoding='utf-8', index=False)